In [1]:
from json import JSONDecoder
from read_json import read_JSON_as_list, fetch_objects, fetch_connections

In [2]:
def read_JSON_as_list(filename):
    def parse_pairs(pairs):
        return pairs

    decoder = JSONDecoder(object_pairs_hook=parse_pairs)
    with open(filename) as json_file:
        file_content = json_file.read()
        data = decoder.decode(file_content)
        
    return data

In [3]:
data = read_JSON_as_list('./JSON/mental_imagery_extended.json')

In [5]:
# objects = get_all_objects(data)
objects = fetch_objects(data)

In [6]:
''' 
data[1][1] has the value for 'steps' --> a list of the different objects 
(except for the ones in groups)
e.g. data[1][1][0] is a NeuralField, its first element is the type of object,
i.e. in this case the string 'cedar.dynamics.NeuralField', its second element
contains the parameters and their values
'''

print(objects[0][0], "\n")

print(objects[0][1], "\n")

cedar.dynamics.NeuralField 

[('name', 'Above Memory'), ('activation as output', 'false'), ('discrete metric (workaround)', 'false'), ('update stepIcon according to output', 'true'), ('threshold for updating the stepIcon', '0.80000000000000004'), ('dimensionality', '0'), ('sizes', ''), ('time scale', '100'), ('resting level', '-5'), ('input noise gain', '0.10000000000000001'), ('sigmoid', [('type', 'cedar.aux.math.AbsSigmoid'), ('threshold', '0'), ('beta', '100')]), ('global inhibition', '-0.01'), ('lateral kernels', [('cedar.aux.kernel.Box', [('dimensionality', '1'), ('anchor', ['0']), ('amplitude', '6.0999999999999996'), ('widths', ['2'])])]), ('lateral kernel convolution', [('borderType', 'Zero'), ('mode', 'Same'), ('engine', [('type', 'cedar.aux.conv.OpenCV')]), ('alternate even kernel center', 'false')]), ('noise correlation kernel', [('dimensionality', '1'), ('anchor', ['0']), ('amplitude', '0'), ('sigmas', ['3']), ('normalize', 'true'), ('shifts', ['0']), ('limit', '5')]), ('com

In [7]:
# check if I have all objects now by checking if the numbers are the same as
# in the json file
object_counts = {}

for obj in objects:
    if obj[0] not in object_counts.keys():
        object_counts[obj[0]] = 1
    else:
        object_counts[obj[0]] += 1
        
for key in object_counts:
    print('Number of %s: %i' %(key, object_counts[key]))

# --> Numbers check out

Number of cedar.dynamics.NeuralField: 90
Number of cedar.processing.sources.Boost: 23
Number of cedar.processing.ComponentMultiply: 52
Number of cedar.processing.sources.ConstMatrix: 8
Number of cedar.processing.steps.Convolution: 4
Number of cedar.processing.Flip: 4
Number of cedar.processing.sources.GaussInput: 16
Number of cedar.processing.Projection: 38
Number of cedar.processing.sources.SpatialTemplate: 24
Number of cedar.processing.StaticGain: 166


# Get all connections
For the groups the connections to and from the input nodes of the groups and output nodes of the groups have to be handled seperately, since these should be replaced by direct connections between the instances.

The names of the input and output nodes for groups are in the group element "connectors". If they have the value "true" they are input nodes, if the have the value "false" they are output nodes. In connections to input nodes of a group the target name is of the form: group_name.input_node_name


In [10]:
all_connections = fetch_connections(data)

In [15]:
len(all_connections)

669

# Test simpler JSON file
To be sure that everything works correctly I added a small test architecture to the JSON files. It includes all scenarios of different group input and output connections, but with only a few objects to stay on top of things. 

In [16]:
test_arc = read_JSON_as_list('./JSON/test_architecture.json')
test_objects = fetch_objects(test_arc)
test_connections = fetch_connections(test_arc)

In [17]:
for obj in test_objects:
    print(obj[1][0][1])

Linear Spatial Template
Projection
Projection 2
Spatial Template
Group 1.Component Multiply
Group 1.Neural Field
Group 1.Static Gain
Group 2.Gauss Input
Group 2.Neural Field 2


In [18]:
test_connections

[[('source', 'Group 2.Neural Field 2.sigmoided activation'),
  ('target', 'Projection.input')],
 [('source', 'Group 2.Neural Field 2.sigmoided activation'),
  ('target', 'Projection 2.input')],
 [('source', 'Linear Spatial Template.spatial pattern'),
  ('target', 'Group 1.Component Multiply.operands')],
 [('source', 'Spatial Template.spatial pattern'),
  ('target', 'Group 1.Component Multiply.operands')],
 [('source', 'Spatial Template.spatial pattern'),
  ('target', 'Group 1.Static Gain.input')],
 [('source', 'Group 1.Neural Field.sigmoided activation'),
  ('target', 'Group 2.Neural Field 2.input')],
 [('source', 'Group 1.Component Multiply.product'),
  ('target', 'Group 1.Neural Field.input')],
 [('source', 'Group 1.Static Gain.output'),
  ('target', 'Group 1.Neural Field.input')],
 [('source', 'Group 2.Gauss Input.Gauss input'),
  ('target', 'Group 2.Neural Field 2.input')]]